# Chapter 4: ARIMA

## Seasonal Decompose
You can think of a time series as being composed of trend, seasonal, and residual components. This can be a good way to think about the data when you go about modeling it. If you know the period of the time series, you can decompose it into these components.

In this exercise, you will decompose a time series showing the monthly milk production per cow in the USA. This will give you a clearer picture of the trend and the seasonal cycle. Since the data is monthly, you will guess that the seasonality might be 12 time periods; however, this won't always be the case.

The milk production time series has been loaded into the DataFrame `milk_production` and is available in your environment.

In [ ]:
# Import seasonal decompose
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt

# Perform additive decomposition
decomp = seasonal_decompose(milk_production['pounds_per_cow'], period=12)

# Plot decomposition
decomp.plot()
plt.show()

## Seasonal ACF and PACF
Below is a time series showing the estimated number of water consumers in London. By eye, you can't see any obvious seasonal pattern; however, your eyes aren't the best tools you have.

In this exercise, you will use the ACF and PACF to test this data for seasonality. You can see from the plot above that the time series isn't stationary, so you should probably detrend it. You will detrend it by subtracting the moving average. Remember that you could use a window size of any value bigger than the likely period.

The `plot_acf()` function has been imported, and the time series has been loaded in as `water`. Plot the ACF of the `water_consumers` column of the time series up to 25 lags.

In [ ]:
# Create figure and subplot
fig, ax1 = plt.subplots()

# Plot the ACF on ax1
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(water['water_consumers'], lags=25, zero=False, ax=ax1)

# Show figure
plt.show()

### Detrending by Subtracting the Moving Average

In [ ]:
# Subtract the rolling mean
water_2 = water - water.rolling(15).mean()

# Drop the NaN values
water_2 = water_2.dropna()

# Create figure and subplots
fig, ax1 = plt.subplots()

# Plot the ACF
plot_acf(water_2['water_consumers'], lags=25, zero=False, ax=ax1)

# Show figure
plt.show()

## Fitting SARIMA Models
Fitting SARIMA models is the beginning of the end of this journey into time series modeling.

It is important that you get to know your way around the SARIMA model orders, and that's what you will focus on here.

In this exercise, you will practice fitting different SARIMA models to a set of time series.

The time series DataFrames `df1`, `df2`, and `df3` and the SARIMAX model class are available in your environment.

In [ ]:
# Create a SARIMAX model
from statsmodels.tsa.statespace.sarimax import SARIMAX
model = SARIMAX(df1, order=(1,0,0), seasonal_order=(1,1,0,7))

# Fit the model
results = model.fit()

# Print the results summary
print(results.summary())

In [ ]:
# Create a SARIMAX model
model = SARIMAX(df2, order=(2,1,1), seasonal_order=(1,0,0,4))

# Fit the model
results = model.fit()

# Print the results summary
print(results.summary())

In [ ]:
# Create a SARIMAX model
model = SARIMAX(df3, order=(1,1,0), seasonal_order=(0,1,1,12))

# Fit the model
results = model.fit()

# Print the results summary
print(results.summary())

# Choosing SARIMA order
In this exercise you will find the appropriate model order for a new set of time series. This is monthly series of the number of employed persons in Australia (in thousands). The seasonal period of this time series is 12 months.

You will create non-seasonal and seasonal ACF and PACF plots and use the table below to choose the appropriate model orders.

AR(p)	MA(q)	ARMA(p,q)
ACF	Tails off	Cuts off after lag q	Tails off
PACF	Cuts off after lag p	Tails off	Tails off
The DataFrame aus_employment and the functions plot_acf() and plot_pacf() are available in your environment.

Note that you can take multiple differences of a DataFrame using df.diff(n1).diff(n2).
Take the first order difference and the seasonal difference of the aus_employment and drop the NaN values. The seasonal period is 12 months.

In [ ]:
# Take the first and seasonal differences and drop NaNs
aus_employment_diff = aus_employment.diff().diff(12).dropna()

Plot the ACF and PACF of aus_employment_diff up to 11 lags.
# Create the figure
fig, (ax1, ax2) = plt.subplots(2,1,figsize=(8,6))

# Plot the ACF on ax1
plot_acf(aus_employment_diff, lags=11, zero=False, ax=ax1)

# Plot the PACF on ax2
plot_pacf(aus_employment_diff, lags=11, zero=False, ax=ax2)

plt.show()

In [ ]:
# Make list of lags
lags = [12, 24, 36, 48, 60]

# Create the figure
fig, (ax1, ax2) = plt.subplots(2,1,figsize=(8,6))

# Plot the ACF on ax1
plot_acf(aus_employment_diff, lags=lags, ax=ax1)

# Plot the PACF on ax2
plot_pacf(aus_employment_diff, lags=lags, ax=ax2)

plt.show()

#SARIMA vs ARIMA forecasts
In this exercise, you will see the effect of using a SARIMA model instead of an ARIMA model on your forecasts of seasonal time series.

Two models, an ARIMA(3,1,2) and a SARIMA(0,1,1)(1,1,1), have been fit to the Wisconsin employment time series. These were the best ARIMA model and the best SARIMA model available according to the AIC.

In the exercise you will use these two models to make dynamic future forecast for 25 months and plot these predictions alongside held out data for this period, wisconsin_test.

The fitted ARIMA results object and the fitted SARIMA results object are available in your environment as arima_results and sarima_results.

Create a forecast object, called arima_pred, for the ARIMA model to forecast the next 25 steps after the end of the training data.
Extract the forecast .predicted_mean attribute from arima_pred and assign it to arima_mean.
Repeat the above two steps for the SARIMA model.
Plot the SARIMA and ARIMA forecasts and the held out data wisconsin_test.

Remember that you can use the fitted results object's .get_forecast() to create a forecast object.
To forecast the next N steps you need to pass steps=N into the .get_forecast() method.
The mean forecast prediction is stored in the .predicted_mean attribute of the forecast object.


In [ ]:
# Create ARIMA mean forecast
arima_pred = arima_results.get_forecast(steps=25)
arima_mean = arima_pred.predicted_mean

# Create SARIMA mean forecast
sarima_pred = sarima_results.get_forecast(steps=25)
sarima_mean = sarima_pred.predicted_mean

# Plot mean ARIMA and SARIMA predictions and observed
plt.plot(dates, sarima_mean, label='SARIMA')
plt.plot(dates, arima_mean, label='ARIMA')
plt.plot(wisconsin_test, label='observed')
plt.legend()
plt.show()

## Automated Model Selection
The `pmdarima` package is a powerful tool to help you choose the model orders. You can use the information you already have from the identification step to narrow down the model orders that you choose by automation.

Remember, although automation is powerful, it can sometimes make mistakes that you wouldn't. It is hard to guess how the input data could be imperfect and affect the test scores.In this exercise you will use the pmdarima package to automatically choose model orders for some time series datasets.


In [ ]:
# Import pmdarima
import pmdarima as pm

# Create auto_arima model
#Model the time series df1 with period 7 days and set first order seasonal differencing and no non-seasonal differencing.
#Use the auto_arima() function from the pmdarima package to fit the data.Make sure you don't mix up the seasonal differencing D and the non-seasonal differencing d.
model1 = pm.auto_arima(df1, seasonal=True, m=7, d=0, D=1, max_p=2, max_q=2, trace=True, error_action='ignore', suppress_warnings=True)

# Print model summary
print(model1.summary())

Create a model to fit df2. Set the non-seasonal differencing to 1, the trend to a constant and set no seasonality.


In [ ]:
# Create model
model2 = pm.auto_arima(df2,
                      seasonal=False,
                      d=1,
                      trend='c',
                 	  max_p=2, max_q=2,
                      trace=True,
                      error_action='ignore',
                      suppress_warnings=True)

# Print model summary
print(model2.summary())

Fit a SARIMAX(p,1,q)(P,1,Q) model to the data setting start_p, start_q, max_p, max_q, max_P and max_Q to 1.


In [ ]:
# Create model for SARIMAX(p,1,q)(P,1,Q)7
model3 = pm.auto_arima(df3,
                      seasonal=True, m=7,
                      d=1, D=1,
                      start_p=1, start_q=1,
                      max_p=1, max_q=1,
                      max_P=1, max_Q=1,
                      trace=True,
                      error_action='ignore',
                      suppress_warnings=True)

# Print model summary
print(model3.summary())

## Saving and Updating Models
Once you have gone through the steps of the Box-Jenkins method and arrived at a model you are happy with, you will want to be able to save that model and also to incorporate new measurements when they are available. This is a key part of putting the model into production.In this exercise you will save a freshly trained model to disk, then reload it to update it with new data.
The model is available in your environment as model.
Import the joblib package and use it to save the model to "candy_model.pkl".

To save the model use the dump() function from the joblib package.
The arguments to the dump() function are the model and the filename.

In [ ]:
# Import joblib
import joblib

# Set model name
filename = 'candy_model.pkl'

# Pickle it
joblib.dump(model, filename)

In [ ]:
# Load the model back in
loaded_model = joblib.load(filename)

# Update the model
loaded_model.update(df_new)

#SARIMA model diagnostics
Usually the next step would be to find the order of differencing and other model orders. However, this time it's already been done for you. The time series is best fit by a SARIMA(1, 1, 1)(0, 1, 1) model with an added constant.

In this exercise you will make sure that this is a good model by first fitting it using the SARIMAX class and going through the normal model diagnostics procedure.

The DataFrame, co2, and the SARIMAX model class are available in your environment.

Fit a SARIMA(1, 1, 1)(0, 1, 1) model to the data and set the trend to constant.

In [ ]:
# Import model class
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Create model object
model = SARIMAX(co2,
                order=(1, 1, 1),
                seasonal_order=(0, 1, 1, 12),
                trend='c')
# Fit model
results = model.fit()

Create the common diagnostics plots for the results object.


In [ ]:
# Plot common diagnostics
results.plot_diagnostics()
plt.show()

#SARIMA forecast
In the previous exercise you confirmed that a SARIMA  x  model was a good fit to the CO time series by using diagnostic checking.

Now its time to put this model into practice to make future forecasts. Climate scientists tell us that we have until 2030 to drastically reduce our CO emissions or we will face major societal challenges.

In this exercise, you will forecast the CO time series up to the year 2030 to find the CO levels if we continue emitting as usual.

The trained model results object is available in your environment as results.

Create a forecast object for the next 136 steps - the number of months until Jan 2030.
Assign the .predicted_mean of the forecast to the variable mean.
Compute the confidence intervals and assign this DataFrame to the variable conf_int.

In [ ]:
    # Create forecast object
    forecast_object = results.get_forecast(steps=136)

    # Extract prediction mean
    mean = forecast_object.predicted_mean

    # Extract the confidence intervals
    conf_int = forecast_object.conf_int()

    # Extract the forecast dates
    dates = mean.index

Plot the mean predictions against the dates.
Shade the area between the values in the first two columns of DataFrame conf_int using dates as the x-axis values.

conf_int is a DataFrame. The lower limits are in the zeroth column conf_int.iloc[:,0] and the higher limits are in the first column.
You will need to pass the dates, the lower limits and the upper limits into the pyplot fill_between() function.

In [ ]:
plt.figure()

# Plot past CO2 levels
plt.plot(co2.index, co2, label='past')

# Plot the prediction means as line
plt.plot(dates, mean, label='predicted')

# Shade between the confidence intervals
plt.fill_between(dates, conf_int.iloc[:,0], conf_int.iloc[:,1], alpha=0.2)

# Plot legend and show figure
plt.legend()
plt.show()

In [ ]:
# Print last predicted mean
print(mean.iloc[-1])

# Print last confidence interval
print(conf_int.iloc[-1])